# Sprawozdanie 2
## Jakub Ciągło (275986), Mateusz Ćwiek (276011)

___
# Importy

In [5]:
import scipy.stats as stats
import numpy as np
import pandas as pd

___
# Część 1
## Zadanie 1

___
## Zadanie 2

In [12]:
def poziomy_krytyczne(x, p_0):
    k = len(p_0)
    n = sum(x)
    
    pearson_stat = sum(((x[i] - n * p_0[i]) ** 2) / (n * p_0[i]) for i in range(k))

    nw_stat = 2 * sum(x[i] * np.log(x[i] / (n * p_0[i])) for i in range(k) if x[i] > 0)
    
    p_value_pearson = 1 - stats.chi2.cdf(pearson_stat, k - 1)
    p_value_nw = 1 - stats.chi2.cdf(nw_stat, k - 1)
    
    return float(p_value_pearson), float(p_value_nw)

x = [20, 20, 20, 20]
p = [0.25, 0.25, 0.25, 0.25]
print(poziomy_krytyczne(x, p))

(1.0, 1.0)


___
## Zadanie 3

In [ ]:
alfa = 0.05
df = pd.read_csv('ankieta.csv', encoding='Latin2', sep=';')

df_produktowy = df[df['DZIAŁ'] == 'PD']

odpowiedzi = [-2, -1, 0, 1, 2]
n = df_produktowy['PYT_1'].value_counts().reindex(odpowiedzi, fill_value=0).tolist()
print('Liczności odpowiedzi:', n)

p_0 = [1/len(n) for _ in range(len(n))] # w hipotezie zakładamy, że prawdopodobieństwa są równe
p_value_pearson, p_value_nw = poziomy_krytyczne(n,p_0)

p_value_pearson, p_value_nw = poziomy_krytyczne(n, p_0)

print(f'Statystyka chi² Pearsona (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_pearson}')
if p_value_pearson > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

print(f'\nStatystyka chi² największej wiarygodności (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_nw}')
if p_value_nw > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

Liczności odpowiedzi: [9, 10, 17, 51, 11]
Statystyka chi² Pearsona (dla poziomu istotności α = 0.05):
  p-wartość = 2.757793993168889e-13
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)

Statystyka chi² największej wiarygodności (dla poziomu istotności α = 0.05):
  p-wartość = 1.0701994845874196e-10
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)
